In [1]:
! pip install tensorflow pandas scikit-learn


Defaulting to user installation because normal site-packages is not writeable


In [10]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("./new3.csv")

# Example for encoding the 'aggr_full_name' and 'full_name' columns
label_encoder = LabelEncoder()

df['aggr_full_name_encoded'] = label_encoder.fit_transform(df['aggr_full_name'])
df['full_name_encoded'] = label_encoder.fit_transform(df['full_name'])

# Extract features and labels
X = df[['aggr_full_name_encoded', 'full_name_encoded']].values
y = df['sim_sco_full_name'].values.astype(float)  # Ensure the labels are float

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate

# Define inputs
input1 = Input(shape=(1,))
input2 = Input(shape=(1,))

# Combine inputs
combined = concatenate([input1, input2])

# Define layers
x = Dense(10, activation='relu')(combined)
output = Dense(1, activation='sigmoid')(x)

# Create model
model = Model(inputs=[input1, input2], outputs=output)

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit([X_train[:, 0], X_train[:, 1]], y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.0000e+00 - loss: -1164.4977 - val_accuracy: 0.0000e+00 - val_loss: -1332.5653
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0000e+00 - loss: -1173.1200 - val_accuracy: 0.0000e+00 - val_loss: -1358.2130
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.0000e+00 - loss: -1183.6584 - val_accuracy: 0.0000e+00 - val_loss: -1384.0057
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: -1228.0081 - val_accuracy: 0.0000e+00 - val_loss: -1409.9695
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.0000e+00 - loss: -1263.6293 - val_accuracy: 0.0000e+00 - val_loss: -1436.0643
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0000e+00 - loss: -1257.8790 - val_accuracy: 0.0000e+00 - val_loss: -1462.2584
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.0000e+00 - loss: -1303.1455 - val_accuracy: 0.0000e+00 - val_loss: -1488.6427
Epoch 8/10
2

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Concatenate

# Load the dataset
df = pd.read_csv('./new3.csv')

# Print the column names to verify
print(df.columns)

# Ensure correct column names
# Adjust column names based on your CSV file
X = df[['aggr_full_name', 'full_name']].values
y = df['sim_sco_full_name'].values

# Tokenizer setup
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(df['aggr_full_name'].tolist() + df['full_name'].tolist())

# Convert names to sequences
sequences1 = tokenizer.texts_to_sequences(df['aggr_full_name'].tolist())
sequences2 = tokenizer.texts_to_sequences(df['full_name'].tolist())

# Determine the maximum length
maxlen = max(max(len(seq) for seq in sequences1), max(len(seq) for seq in sequences2))

# Pad the sequences
X1 = tf.keras.preprocessing.sequence.pad_sequences(sequences1, maxlen=maxlen)
X2 = tf.keras.preprocessing.sequence.pad_sequences(sequences2, maxlen=maxlen)

# Split the data
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

# Define input layers
input1 = Input(shape=(maxlen,))
input2 = Input(shape=(maxlen,))

# Embedding layer
embedding_dim = 50
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=maxlen)

# Process each input
encoded1 = embedding_layer(input1)
encoded2 = embedding_layer(input2)

# BiLSTM layers
shared_lstm = Bidirectional(LSTM(64))
output1 = shared_lstm(encoded1)
output2 = shared_lstm(encoded2)

# Concatenate the outputs
merged = Concatenate()([output1, output2])

# Fully connected layer
dense = Dense(64, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(dense)

# Define the model
model = Model(inputs=[input1, input2], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit([X1_train, X2_train], y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X1_test, X2_test], y_test)
print(f'Test Accuracy: {accuracy}')

# Save the model
model.save('./name_matching_model.h5')

# Example names to match
new_names = [('John Doe', 'Jon Doe'), ('Alice Smith', 'Alicia Smith')]

# Convert and pad the new names
new_seq1 = tokenizer.texts_to_sequences([name[0] for name in new_names])
new_seq2 = tokenizer.texts_to_sequences([name[1] for name in new_names])
new_X1 = tf.keras.preprocessing.sequence.pad_sequences(new_seq1, maxlen=maxlen)
new_X2 = tf.keras.preprocessing.sequence.pad_sequences(new_seq2, maxlen=maxlen)

# Make predictions
predictions = model.predict([new_X1, new_X2])
print(predictions)


C:\Users\subasini\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Index(['loc_id', 'loc_name', 'aggr_order_channel', 'aggr_full_name',
       'aggr_soundex_score', 'channel', 'full_name', 'magil_soundex_score',
       'maghil_phone', 'match_score', 'first_name', 'last_name',
       'agg_first_name', 'agg_last_name', 'sim_sco_full_name',
       'sim_sco_first_name', 'sim_sco_last_name'],
      dtype='object')


C:\Users\subasini\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 32, 50)    │      1,400 │ input_layer[0][0… │
│ (Embedding)         │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128)       │     58,880 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     16,448 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 76,793 (299.97 KB)

 Trainable params: 76,793 (299.97 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 652ms/step - accuracy: 0.0000e+00 - loss: 1.6077 - val_accuracy: 0.0000e+00 - val_loss: -3.4320
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0000e+00 - loss: -4.3114 - val_accuracy: 0.0000e+00 - val_loss: -11.0766
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0000e+00 - loss: -12.6410 - val_accuracy: 0.0000e+00 - val_loss: -25.3592
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0000e+00 - loss: -28.2631 - val_accuracy: 0.0000e+00 - val_loss: -51.6353
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.0000e+00 - loss: -56.3180 - val_accuracy: 0.0000e+00 - val_loss: -94.9940
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.0000e+00 - loss: -102.1339 - val_accuracy: 0.0000e+00 - val_loss: -160.1724
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.0000e+00 - loss: -169.7659 - val_accuracy: 0.0000e+00 - val_loss: -252.5485
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6

Test Accuracy: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
[[0.99999976]
 [0.9999998 ]]


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import numpy as np

# Load the model
model = load_model('F:/Task/new/name_matching_model.h5')

# Load your data
df = pd.read_csv('path/to/your/data.csv')

# Initialize the label encoder
label_encoder = LabelEncoder()

# Encode the names
df['aggr_full_name_encoded'] = label_encoder.fit_transform(df['aggr_full_name'])
df['full_name_encoded'] = label_encoder.fit_transform(df['full_name'])

# Prepare the input data for prediction
X1 = df['aggr_full_name_encoded'].values
X2 = df['full_name_encoded'].values

# Make sure the input shape is correct
X1 = X1.reshape(-1, 1)
X2 = X2.reshape(-1, 1)

# Predict the similarity scores
similarity_scores = model.predict([X1, X2])

# Add the similarity scores to a new column in the DataFrame
df['similarity_score'] = similarity_scores

# Save the DataFrame with similarity scores
df.to_csv('path/to/your/updated_data.csv', index=False)
